# Custom Training with YOLOv8

In this lab exercise, we will learn to train a custom YOLOv8 model to perform object detection. To do so we will take the following steps:

* Gather a dataset of images and label our dataset
* Export our dataset to YOLOv8 format
* Train YOLOv8 to recognize the objects in our dataset
* Evaluate our YOLOv8 model's performance
* Run inference to test our deployed model



![](https://uploads-ssl.webflow.com/5f6bc60e665f54545a1e52a5/615627e5824c9c6195abfda9_computer-vision-cycle.png)

# Step 1: Install Requirements

In [ ]:
!pip install -q ultralytics
!pip install -q roboflow
!pip install -q gradio

# Step 2: Dataset

In this exercise, we will use an existing annotated dataset for our training. This dataset, the apples dataset is available from [Roboflow Universe](https://universe.roboflow.com/) which is a collection of open source computer vision datasets.

You can read more about the apples dataset [here](https://universe.roboflow.com/roboflow-100/apples-fvpl5). The dataset consists of both normal apple, and damaged apple.

We can use Roboflow to also convert the dataset format, if the dataset is not already in YOLOv8 format. Roboflow supports many different [annotation formats](https://roboflow.com/formats)

You can also use Roboflow to annotate your own images if they are not already annotated.

**Annotation using Roboflow**

![](https://roboflow-darknet.s3.us-east-2.amazonaws.com/roboflow-annotate.gif)


To download the dataset from Roboflow, you will need a Roboflow account. You can register for a free account [here](https://app.roboflow.com/login).

You can then get your API key from the account you created. Go to **Account -> Settings -> New Workspace -> Roboflow API**.  You should be able to see your API key there. Click on the copy button for Private API Key.

<img src="https://raw.githubusercontent.com/nyp-sit/nypi/main/resources/roboflow_api_key.png" width=500/>

In [ ]:
from roboflow import Roboflow

# rf = Roboflow(api_key="?????")

rf = Roboflow(api_key="cBYcnpRQaCFSCkiX1Y7R")
project = rf.workspace("roboflow-100").project("apples-fvpl5")
dataset = project.version(2).download("yolov8")

By default, ultralytics training script expects your dataset to be in the *datasets* folder. So we will move the downloaded dataset to the *datasets* subfolder, and we also need to change directory in the *datasets* folder to start the train script.

In [ ]:
%mkdir datasets
%mv apples-2 datasets
%cd datasets
!yolo detect train data=apples-2/data.yaml model=yolov8n.pt epochs=15
%cd ..

# Visualize Training loss and Performance metrics

Training losses and performance metrics are saved as Tensorboard logging events.  You can view the metrics using the tensorboard plugin by running the cell below.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "datasets/runs/"
%load_ext tensorboard
%tensorboard --logdir datasets/runs/detect/train

# Run validation

The following codes run the model against the validation dataset and display the validation results.

In [ ]:
%cd datasets
!yolo val model=/content/datasets/runs/detect/train/weights/best.pt data=apples-2/data.yaml
%cd ../

# Running inference on Test Image

Now that our model is trained. We can do inference using our best model checkpoint which is located at `datasets/runs/detect/train/weights/best.pt`

The results contains the numpy array of the image with detection boxes. channel of the returned image array is in the format of BGR (blue, green, red). If you want to render it using PIL or matplotlib, you need to convert it to RGB format first.

In [ ]:
# Let's define a function to do the prediction
from PIL import Image
from ultralytics import YOLO

def predict(img):
    '''
    Parameters
    ----------
    img: a PIL Image

    Returns
    -------
    PIL image with bounding boxes
    '''


    model = YOLO('datasets/runs/detect/train/weights/best.pt')
    results = model.predict(source=img, save=True, conf=0.4)
    img_bgr = results[0].plot(conf=True, font_size=10)
    img_rgb = img_bgr[:, :, [2, 1, 0]]
    image_detected = Image.fromarray(img_rgb)

    return image_detected


In [ ]:
test_image = '/content/datasets/apples-2/test/images/apple--179-_jpg.rf.a70c7896db8bfdb74e35d9aed5049f54.jpg'
test_image = Image.open(test_image)
image_detected = predict(test_image)
image_detected.show()

# Demo App

Now let's build a demo app that allows us to upload a picture of apple, and it will return the image with bounding boxes.

In [ ]:
import gradio as gr

gr.Interface(fn=predict,
             inputs=gr.Image(type="pil"),
             outputs=gr.Image(type="pil")
             ).launch(share=True)

# Exercise

You will be given a new dataset that has not been annotated. You will use Roboflow to annotate the images and download the dataset in YOLOv8 format. You can then train a new model as before.